In [2]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langchain_community.tools import ShellTool
from langgraph.prebuilt import ToolNode
from typing import TypedDict, List
import os

In [ ]:
# Set up Groq LLM
os.environ["GROQ_API_KEY"] = "your-groq-api-key-here"  # Replace with your actual key

llm = ChatGroq(
    model="llama3-8b-8192",  # or another model
    temperature=0.1
)

In [ ]:
# Define the state
class CodeReviewState(TypedDict):
    code: str
    feedback: List[str]
    issues: List[str]
    suggestions: List[str]

In [ ]:
# Define nodes
def analyze_code(state: CodeReviewState):
    code = state["code"]
    prompt = f"""
    Analyze the following code for potential issues, bugs, and improvements.
    Code:
    {code}
    
    Provide a list of issues and suggestions.
    """
    response = llm.invoke([SystemMessage(content="You are a code review expert."), HumanMessage(content=prompt)])
    analysis = response.content
    
    # Parse analysis into issues and suggestions (simple split)
    lines = analysis.split('\n')
    issues = [line for line in lines if 'issue' in line.lower() or 'bug' in line.lower()]
    suggestions = [line for line in lines if 'suggest' in line.lower() or 'improve' in line.lower()]
    
    return {"issues": issues, "suggestions": suggestions}

def provide_feedback(state: CodeReviewState):
    issues = state["issues"]
    suggestions = state["suggestions"]
    feedback = f"Issues: {issues}\nSuggestions: {suggestions}"
    return {"feedback": [feedback]}

In [ ]:
# Build the graph
graph = StateGraph(CodeReviewState)

graph.add_node("analyze", analyze_code)
graph.add_node("feedback", provide_feedback)

graph.add_edge(START, "analyze")
graph.add_edge("analyze", "feedback")
graph.add_edge("feedback", END)

code_review_agent = graph.compile()

In [ ]:
# Test the agent
sample_code = """
def hello_world():
    print("Hello, World!")
"""

result = code_review_agent.invoke({"code": sample_code, "feedback": [], "issues": [], "suggestions": []})
print(result)

In [1]:
print("hi")

hi
